In [3]:
import torchvision.models as models
resnet18 = models.resnet18(weights='IMAGENET1K_V1')

In [1]:
import sys
sys.path.append("..")

import torch
from torchvision import transforms
from torch.utils.data import DataLoader, ConcatDataset
from datasets.backdoor_dataset import CIFAR10M,CustomDataset_224,CIFAR10Mem
import numpy as np
from datasets.bd_dataset_imagenet_filter import BadEncoderDataset


def cifar10_dataloader():
    batch_size=128
    classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(32),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
        ])

    clean_transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
                ])

    memory_data = CIFAR10M(numpy_file='../data/cifar10/train.npz', class_type=classes, transform=train_transform,transform2=clean_transform)
    train_loader = DataLoader(memory_data, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

    test_data = CIFAR10M(numpy_file='../data/cifar10/test.npz', class_type=classes, transform=train_transform,transform2=clean_transform)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

    return train_loader,test_loader


In [15]:
import sys
sys.path.append('../')
from datasets.backdoor_dataset import *
import numpy as np
import torch
from datasets.trans import *
import argparse
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

test_transform_cifar10 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    ])
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    ])
class BadEncoderTestBackdoor(Dataset):

    def __init__(self, numpy_file, trigger_file, reference_label, aug_transform=None, clean_transform=None):
        """
        Args:
            numpy_file (string): Path to the numpy file.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.input_array = np.load(numpy_file)
        self.data = self.input_array['x']
        self.targets = self.input_array['y']


        self.trigger_input_array = np.load(trigger_file)

        self.trigger_patch_list = self.trigger_input_array['t']
        self.trigger_mask_list = self.trigger_input_array['tm']

        self.target_class = reference_label

        self.test_transform = clean_transform
        self.aug_transform = aug_transform

        # state_dict = torch.load(trigger_file, map_location=torch.device('cpu'))
        # self.net = U_Net_tiny(img_ch=3,output_ch=3)
        # self.net.load_state_dict(state_dict['model_state_dict'])
        # self.net=self.net.eval()

        # self.filter = torch.load('trigger/filter.pt', map_location=torch.device('cpu'))

    def __getitem__(self,index):
        img = copy.deepcopy(self.data[index])

        ########################
        img_ = Image.fromarray(img)
        img_clean =self.test_transform(img_)
        img_clean_aug = self.aug_transform(img_)

        ###########################
        ### for ins filter only ###

        image_pil = Image.fromarray(img)
        filtered_image_pil = pilgram.xpro2(image_pil)
        img_backdoor_ins =self.test_transform(filtered_image_pil)
        img_backdoor_ins_aug = self.aug_transform(filtered_image_pil)

        ###########################

        # img[:] =img * self.trigger_mask_list[0] + self.trigger_patch_list[0][:]
        # img_backdoor_patch =self.test_transform(Image.fromarray(img))



        return img_clean, img_clean_aug, img_backdoor_ins, img_backdoor_ins_aug

    def __len__(self):
        return self.data.shape[0]


args=argparse.Namespace(
    data_dir='../data/cifar10/',
    patch='../trigger/trigger_pt_white_21_10_ap_replace.npz',
    filter='../optimize_filter/trigger/cifar10/2023-12-06-23-41-20/ssim0.9328_psnr22.50_lp0.0291_wd0.603_color11.353.pt',
    reference_file='reference/cifar10/truck.npz',
    reference_label=0,
    shadow_dataset='cifar10'
)
test_data_backdoor = BadEncoderTestBackdoor(numpy_file=args.data_dir+'test.npz', trigger_file=args.patch, reference_label= args.reference_label,  aug_transform=train_transform,clean_transform=test_transform_cifar10)
test_data_backdoor=DataLoader(test_data_backdoor, batch_size=512, shuffle=False, num_workers=2, pin_memory=True)

net = U_Net_tiny(img_ch=3,output_ch=3)
state_dict = torch.load(args.filter, map_location=torch.device('cuda:0'))
net.load_state_dict(state_dict['model_state_dict'])
net=net.cuda().eval()

In [11]:
import torch
from torchvision import models, transforms
from optimize_filter.PyTorch_CIFAR10.cifar10_models.resnet import resnet34
from torch.nn import Identity
import pickle
# 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

backbone = resnet34(pretrained=True)
backbone.fc = Identity()

# 将模型移动到GPU
backbone = backbone.to(device)

backbone.eval() # for evaluation
# 对于每个图像，提取特征
features_patch_all = []
features_ins_all = []
feature_filter_all = []


for img_clean, img_clean_aug, img_backdoor_ins, img_backdoor_ins_aug in test_data_backdoor:

    # 将数据移动到GPU
    img_backdoor_patch = img_backdoor_patch.to(device)
    img_backdoor_ins_aug = img_backdoor_ins_aug.to(device)
    img_backdoor_ins = img_backdoor_ins.to(device)
    img_clean_aug = img_clean_aug.to(device)
    img_clean = img_clean.to(device)

    img_backdoor_filter=net(img_clean_aug)

    # 计算残差
    residual_patch = (img_backdoor_patch - img_clean)
    residual_ins = (img_backdoor_ins - img_clean)
    residual_filter = (img_backdoor_filter - img_clean)
    # 确保在没有梯度的情况下执行前向传播以提取特征
    with torch.no_grad():
        # 使用backbone提取原始特征

        # features_patch = backbone(img_backdoor_patch)
        # features_ins = backbone(img_backdoor_ins)
        # features_filter = backbone(img_backdoor_filter)

        # 使用backbone提取残差特征
        features_patch = backbone(residual_patch)
        features_ins = backbone(residual_ins)
        features_filter = backbone(residual_filter)


    features_patch_all.append(features_patch)
    features_ins_all.append(features_ins)
    feature_filter_all.append(features_filter)
# 将所有的特征连接成一个完整的张量
features_patch_all = torch.cat(features_patch_all, dim=0)
features_ins_all = torch.cat(features_ins_all, dim=0)
feature_filter_all = torch.cat(feature_filter_all, dim=0)

features = {'patch': features_patch_all, 'ins': features_ins_all, 'filter': feature_filter_all}
out = 'feature_bank/features_residual.pkl'

with open(out, 'wb') as f:
    pickle.dump(features, f)


In [22]:
import torch
from torchvision import models, transforms
from optimize_filter.PyTorch_CIFAR10.cifar10_models.resnet import resnet50
from torch.nn import Identity
import pickle
from util import clamp_batch_images

# 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

backbone = resnet50(pretrained=True)
backbone.fc = Identity()

# 将模型移动到GPU
backbone = backbone.to(device)

backbone.eval() # for evaluation
# 对于每个图像，提取特征
features_patch_all = []
features_ins_all = []
features_ins_aug_all = []
features_filter_all = []
features_filter_aug_all = []


for img_clean, img_clean_aug, img_backdoor_ins, img_backdoor_ins_aug in test_data_backdoor:

    # 将数据移动到GPU
    # img_backdoor_patch = img_backdoor_patch.to(device)
    img_backdoor_ins_aug = img_backdoor_ins_aug.to(device)
    img_backdoor_ins = img_backdoor_ins.to(device)

    img_clean_aug = img_clean_aug.to(device)
    img_clean = img_clean.to(device)

    img_backdoor_filter=net(img_clean)
    img_backdoor_filter=clamp_batch_images(img_backdoor_filter,args)
    img_backdoor_filter_aug=net(img_clean_aug)
    img_backdoor_filter_aug=clamp_batch_images(img_backdoor_filter_aug,args)


    with torch.no_grad():
        img_backdoor_ins = (img_backdoor_ins - img_clean)
        img_backdoor_ins_aug = (img_backdoor_ins_aug - img_clean)
        img_backdoor_filter = (img_backdoor_filter - img_clean)
        img_backdoor_filter_aug = (img_backdoor_filter_aug - img_clean)

        # 使用backbone提取残差特征
        features_ins = backbone(img_backdoor_ins)
        features_ins_aug = backbone(img_backdoor_ins_aug)
        features_filter = backbone(img_backdoor_filter)
        features_filter_aug = backbone(img_backdoor_filter_aug)


    features_ins_aug_all.append(features_ins_aug)
    features_ins_all.append(features_ins)
    features_filter_all.append(features_filter)
    features_filter_aug_all.append(features_filter_aug)

# 将所有的特征连接成一个完整的张量
features_ins_aug_all = torch.cat(features_ins_aug_all, dim=0)
features_ins_all = torch.cat(features_ins_all, dim=0)
features_filter_all = torch.cat(features_filter_all, dim=0)
features_filter_aug_all = torch.cat(features_filter_aug_all, dim=0)

features = {'ins': features_ins_all, 'filter': features_filter_all, 'ins_aug': features_ins_aug_all, 'filter_aug': features_filter_aug_all}
out = 'feature_bank/features_residual_aug.pkl'

with open(out, 'wb') as f:
    pickle.dump(features, f)
